In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /g/data/tm70/ml0072/COMMON/git_repos/fork_access-eval-recipes/ml_create_own_datastore

/g/data/tm70/ml0072/COMMON/git_repos/fork_access-eval-recipes/ml_create_own_datastore


In [3]:
import sys
import os
import re
from pprint import pprint
tool_dir = '/g/data/tm70/ml0072/COMMON/git_repos/fork_access-eval-recipes/ml_create_own_datastore'
sys.path.append(tool_dir)
from plot_functions import _read_ryaml

In [4]:
pics_folder = "esmf-profiling"
pics_folder = os.path.join(tool_dir, pics_folder)
os.makedirs(pics_folder, exist_ok=True)

In [5]:
expts_manager_path = '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager'
yaml_input = os.path.join(expts_manager_path, 'Expts_manager-025deg-scaling_performance.yaml')
yamlfile_path = os.path.join(tool_dir,yaml_input)
yaml_input = _read_ryaml(yamlfile_path)
test_rel_path = yaml_input['test_path']
ncpus_list_combo1 = yaml_input['namelists']['cross_block1']['nuopc.runconfig']['PELAYOUT_attributes_combo']
esmf_expt_dir1 = yaml_input['namelists']['cross_block1']['cross_block1_dirs']
esmf_expt_dirs = list(esmf_expt_dir1)
MOM_full_paths = [os.path.join(expts_manager_path, test_rel_path, esmf_expt_dir) for esmf_expt_dir in  esmf_expt_dirs]
MOM_full_paths

['/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_1_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_2_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_3_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_4_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_5_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_6_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_7_test',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_8_test',
 '/g/data/tm70/ml0072/COMMON/git

In [6]:
xxx = '000'
ESMF_path = [f"{MOM_full_path}/archive/output{xxx}" for MOM_full_path in MOM_full_paths]
ESMF_path

['/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_1_test/archive/output000',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_2_test/archive/output000',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_3_test/archive/output000',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_4_test/archive/output000',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_5_test/archive/output000',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_6_test/archive/output000',
 '/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_scaling_performance/pt_7_test/archive/output000',
 '/g/data/tm70/ml0072/COMMO

In [7]:
def list_esmf_files(dir_path,start_prefix,esmf_summary=False, summary_profile = 'ESMF_Profile.summary'):
    files = os.listdir(dir_path)
    if not esmf_summary:
        matching_files = [file for file in files if file.startswith(start_prefix) and file!=summary_profile]
        matching_files.sort(key=lambda x:int(x[len(start_prefix):]))
        matching_files_path = [os.path.join(dir_path, matching_file) for matching_file in matching_files]
    else:
        matching_file = summary_profile
        matching_files_path = os.path.join(dir_path, matching_file)
    return matching_files_path

In [26]:
class RegionNode:
    def __init__(self,name,count,total,self_time,mean,min_time,max_time):
        self.name = name
        self.count = count
        self.total = total
        self.self_time = self_time
        self.mean = mean
        self.min_time = min_time
        self.max_time = max_time
        self.children = []

    def add_child(self,child):
        self.children.append(child)

    def to_dict(self):
        return {
            'name':self.name,
            'count': self.count,
            'total': self.total,
            'self_time': self.self_time,
            'mean': self.mean,
            'min_time': self.min_time,
            'max_time': self.max_time,
            'children': [child.to_dict() for child in self.children]
        }
def tree_to_dict(node):
    return node.to_dict()

In [8]:
def parse_line(line):
    parts = re.split(r'\s{2,}',line.strip())
    name = parts[0]
    count = int(parts[1])
    total = float(parts[2])
    self_time = float(parts[3])
    mean = float(parts[4])
    min_time = float(parts[5])
    max_time = float(parts[6])
    return name, count, total, self_time, mean, min_time, max_time


In [24]:
def build_trees(lines, skip):
    stack = []
    root = None

    for line in lines:
        if not line.strip(): # skip empty lines
            pass
        if line.startswith('  [ESMF]'):
            skip=True
        if skip:
            #print(line)
            indent_level = len(line) - len(line.lstrip()) # determine hierachy
            name, count, total, self_time, mean, min_time, max_time = parse_line(line)
            #print(name, count, total, self_time, mean, min_time, max_time)
            node = RegionNode(name, count, total, self_time, mean, min_time, max_time)
            if not stack: # the first root
                root = node
            else:
                while stack and stack[-1][1] >= indent_level:
                    stack.pop()
                if stack:
                    stack[-1][0].add_child(node)
            stack.append((node,indent_level))
        #print(stack)
    return root

In [34]:
def find_node_value(node, target_name):
    if node.name == target_name:
        return (node.total, node.self_time, node.mean, node.min_time, node.max_time), True  # Return values and a flag indicating match
    for child in node.children:
        result, found = find_node_value(child, target_name)
        if found:
            return result, found
    return (None, None, None, None, None), False


In [55]:
def find_node_value(node, target_hierarchy):
    """Recursively search for a node based on a full hierarchical path."""
    # Split the target hierarchy into parts
    target_parts = target_hierarchy.split('/')

    # If the hierarchy is empty, return None
    if not target_parts:
        return (None, None, None, None, None), False

    # Check if the current node matches the first part in the hierarchy
    if node.name == target_parts[0]:
        # If the full hierarchy is found, check if we need to go deeper
        if len(target_parts) == 1:
            # We found the node at the end of the path
            return (node.total, node.self_time, node.mean, node.min_time, node.max_time), True
        
        # Continue searching in children if more hierarchy exists
        for child in node.children:
            result, found = find_node_value(child, '/'.join(target_parts[1:]))
            if found:
                return result, found

    # Continue searching through children if not found
    for child in node.children:
        result, found = find_node_value(child, target_hierarchy)
        if found:
            return result, found

    return (None, None, None, None, None), False

In [62]:
%%time
start_prefix = 'ESMF_Profile.'
not_include_files = 'ESMF_Profile.summary'
subfiles_path  = list_esmf_files(ESMF_path[0],start_prefix,not_include_files)
#pprint(subfiles_path)
root_all = []
for index, subfile in enumerate(subfiles_path):
    #pprint(subfiles_path[index])
    with open(subfile,'r') as f:
        eff_lines = []
        skip = False
        root = build_trees(f, skip)
        tree_dict = tree_to_dict(root)
        root_all.append(root)
#root_all
#tree_dict

CPU times: user 1.11 s, sys: 240 ms, total: 1.35 s
Wall time: 1.73 s


In [ ]:
find_node_value(root, varname)

In [63]:
def component_time_consumption(varnames, root_all):
    runtime = {}
    for varname in varnames:
        runtime[varname] = [find_node_value(sub_root, varname)[0][4] for sub_root in root_all]
    return runtime

In [65]:
varnames = [
    '[ESMF]',
    # '[ensemble] RunPhase1',
    # '[OCN] RunPhase1',
    # '[ICE] RunPhase1',
    # '[ATM] RunPhase1',
    # '[ROF] RunPhase1',
     '[ESMF]/[ensemble] RunPhase1/[ESM0001] RunPhase1/[MED-TO-OCN] RunPhase1',
    # '[OCN-TO-MED] RunPhase1',
    # '[MED-TO-ICE] RunPhase1',
    # '[ICE-TO-MED] RunPhase1',
    # '[ATM-TO-MED] RunPhase1',
    # '[ROF-TO-MED] RunPhase1',
    # '[ensemble] Init 1',
    # '[MED] IPDv03p7',
]

runtime = component_time_consumption(varnames, root_all)
runtime

{'[ESMF]': [279.5061,
  279.5392,
  279.5319,
  279.5204,
  279.8237,
  279.5183,
  279.9133,
  279.6565,
  279.6916,
  279.8501,
  279.5243,
  279.5142,
  279.5268,
  279.5363,
  279.8487,
  279.5409,
  279.5114,
  279.5197,
  279.5087,
  279.5103,
  279.8906,
  279.6556,
  279.5999,
  279.535,
  279.5166,
  279.5289,
  279.5086,
  279.5386,
  279.5226,
  279.522,
  279.5726,
  279.512,
  279.5438,
  279.5352,
  279.526,
  279.5269,
  279.5077,
  279.5094,
  279.5077,
  279.6507,
  279.5126,
  279.5424,
  279.5117,
  279.5297,
  279.5176,
  279.5336,
  279.5408,
  279.5149,
  279.5236,
  279.5369,
  279.5359,
  279.5436,
  279.5647,
  279.5417,
  279.5402,
  279.5671,
  279.5294,
  279.5151,
  279.5314,
  279.5179,
  279.5443,
  279.5659,
  279.5417,
  279.5136,
  279.5508,
  279.5636,
  279.5223,
  279.5705,
  279.5529,
  279.5578,
  279.5276,
  279.5656,
  279.5684,
  279.5343,
  279.5691,
  279.5304,
  279.5517,
  279.5321,
  279.568,
  279.5334,
  279.5111,
  279.5427,
  279.5208,